# SDP HPSO Scheduling

Last run with Jupyter Notebook 5.0.0 running Python 3.5.2

In [ ]:
# Imports
from __future__ import print_function
import sys
import matplotlib.pyplot as plt
sys.path += ['..']
from sdp_par_model import reports as iapi
from sdp_par_model import evaluate as imp
from sdp_par_model.config import PipelineConfig
from sdp_par_model.parameters.definitions import *
from sdp_par_model.parameters.definitions import Constants as c
import numpy as np
import collections
import warnings
import bisect

%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 8

## Define structures and methods for handling SDP tasks 

In [ ]:
epsilon = 1e-12  # Used for numerical stability (rounding errors)

# Needs some refactoring methinks; idea would be to specify HPSOs instead of "letters". 
hpso_lookup = {'A' : HPSOs.hpso01, 
               'B' : HPSOs.hpso04c,  # TODO: This task non-imaging. Interesting use case.
               'C' : HPSOs.hpso13, 
               'D' : HPSOs.hpso14,
               'E' : HPSOs.hpso15,
               'F' : HPSOs.hpso27,
               'G' : HPSOs.hpso37c}

# The following results map was copied from examples used by Peter Wortmann. It defines values we wish to calculate.
#               Title                      Unit       Default? Sum?             Expression
results_map =[('Total buffer ingest rate','TeraBytes/s',True, False, lambda tp: tp.Rvis_ingest*tp.Nbeam*tp.Npp*tp.Mvis/c.tera),
              ('Working (cache) memory',  'TeraBytes',  True, True,  lambda tp: tp.Mw_cache/c.tera,   ),
              ('Visibility I/O Rate',     'TeraBytes/s',True, True,  lambda tp: tp.Rio/c.tera,        ),
              ('Total Compute Rate',       'PetaFLOP/s', True, True,  lambda tp: tp.Rflop/c.peta,      ),
              ('Comp Req Breakdown ->',   'PetaFLOP/s', True, True,  lambda tp: tp.get_products('Rflop', scale=c.peta), )]
del results_map[4]  # We actually don't care about the breakdown for now; but it is useful to know how to get it


class DataLocation:
    ingest_buffer = "ingestbuffer"
    cold_buffer   = "coldbuffer"
    hot_buffer    = "hotbuffer"
    preserve      = "preserve"

class SDPTask:
    uid         = None  # Unique ID - must be defined at task creation. Used for hashing and equality.
    description = None  # A human-readable description of the task
    t_min_start = None  # Earliest wall clock time (in seconds) that this task can / may start
    prec_task   = None  # Preceding task that needs to complete before this one can start (can be None)
    t_fixed     = None  # fixed minimum duration (in seconds) of this task (e.g. for an observation)
    data_in     = None  # Amount of data (in TB) this task requires to read from (presumably hot) buffer *before* starting
    memsize     = None  # The amount of SDP working memory (in TB) needed to perform this task
    flopcount   = None  # Number of operations (in PetaFLOP) required to complete this task
    data_out    = None  # Amount of data (in TB) this task stores *after* finishing (written to buffer or long-term preserve)
    data_source = None  # Data Location
    data_target = None  # Data Location

    def __init__(self, unique_id, description=None):
        self.uid = unique_id
        self.description = description
    
    def __hash__(self):
        return hash(self.uid)  # Required for using Task objects in sets.
    
    def __eq__(self, other):
        if not isinstance(other, SDPTask):
            return False
        return (self.uid == other.uid)

    def __str__(self):
        s = "SDPTask #%d (type undefined): " % self.uid
        if self.description is not None:
            s = "SDPTask #%d (%s):" % (self.uid, self.description)
        fields = self.__dict__
        for key_string in fields.keys():
            if key_string == 'uid':
                continue  # We already printed the uid
            elif (key_string == 'prec_task') and (isinstance(fields[key_string], SDPTask)):
                value_string = "SDPTask #%d" % fields[key_string].uid  # Prevent recursion
            else:
                value_string = str(fields[key_string])
                
            if len(value_string) > 40:
                value_string = value_string[:40] + "... (truncated)"
            s += "\n %s\t\t= %s" % (key_string, value_string)
        return s
    
    
    def set_param(self, param_name, value, prevent_overwrite=True, require_overwrite=False):
        """
        Method for setting a parameter in a safe way. By default first checks that the value has not already been defined.
        Useful for preventing situations where values may inadvertently be overwritten.

        :param param_name: The name of the parameter/field that needs to be assigned - provided as text
        :param value: the value to be written (as actual data type, i.e. not necessarily text) 
        :param prevent_overwrite: Disallows this value to be overwritten once defined. Default = True.
        :param require_overwrite: Only allows value to be changed if it already exists. Default = False.
        """
        assert isinstance(param_name, str)
        if prevent_overwrite:
            if require_overwrite:
                raise AssertionError("Cannot simultaneously require and prevent overwrite of parameter '%s'" % param_name)

            if hasattr(self, param_name):
                if eval('self.%s == value' % param_name):
                    print('reassigning value for %s = %s' % (param_name, str(value)))  #TODO remove
                    warnings.warn('Inefficiency : reassigning parameter "%s" with same value as before.' % param_name)
                else:
                    try:
                        assert eval('self.%s == None' % param_name)
                    except AssertionError:
                        raise AssertionError("The parameter %s has already been defined and may not be overwritten." % param_name)

        elif require_overwrite and (not hasattr(self, param_name)):
            raise AssertionError("Parameter '%s' is undefined and therefore cannot be assigned" % param_name)

        exec('self.%s = value' % param_name)  # Write the value

            
def task_letters_to_SDPTask_list(letter_sequence, performance_dict):
    """
    Converts a list of task letters into a list of SDPTask objects
    @param letter_sequence : a sequence of HPSOs, defined by Rosie's A..G lettering scheme. TODO: replace by actual HPSO names
    @param performance_dict : a dictionary of computational requirements for each HPSO; these need to be computed only once
    """
    tasks = []
    uid =  -1
    prev_ingest_task = None
    for task_letter in letter_sequence:
        hpso = hpso_lookup[task_letter]
        hpso_subtasks = HPSOs.hpso_subtasks[hpso]
        nr_subtasks = len(hpso_subtasks)
        
        assert nr_subtasks >= 2  # We assume that the tast as *at least* an Ingest and an RCal component
        if not (hpso_subtasks[0] in HPSOs.ingest_subtasks) and (hpso_subtasks[1] in HPSOs.rcal_subtasks):
            # this is assumed true for all HPSOs - hence raising an assertion error if not
            raise AssertionError("Assumption was wrong - some HPSO apparently doesn't involve Ingest + RCal")
        
        # Ingest and Rcal are combined into a a single task object, as they cannot be separated         
        uid += 1  # the unique id of the combined Ingest+Rcal task        
        t = SDPTask(uid, 'Ingest+RCal')
        if prev_ingest_task is not None:
            t.set_param("prec_task", prev_ingest_task) # previous ingest task needs to be complete before this one can start
        t.set_param("t_fixed", performance_dict[hpso]['Tobs'])
        t.set_param("data_source", DataLocation.ingest_buffer)
        t.set_param("data_target", DataLocation.cold_buffer)
        prev_ingest_task = t  # current (ingest+rcal) task remembered for later; subtasks may only start once this completes
        
        # Ingest
        subtask = hpso_subtasks[0]
        data_in = 0 # data is acquired in real time. TODO: How much?
        memsize = performance_dict[hpso][subtask]['cache']
        flopcount = t.t_fixed * performance_dict[hpso][subtask]['compRate']
        data_out = t.t_fixed * performance_dict[hpso][subtask]['ingestRate']  # ingested data to [cold] buffer.
        
        # RCal
        subtask = hpso_subtasks[1]  
        data_in += 0 # data is acquired in real time. TODO: How much?
        memsize += performance_dict[hpso][subtask]['cache']
        flopcount += t.t_fixed * performance_dict[hpso][subtask]['compRate']
        data_out  += 0 # What output does RCal generate? Just set it to zero for now. Are ingestRate and visRate relevant?
        
        # Set the aggregated task parameters that were computed
        t.set_param("data_in", data_in)
        t.set_param("memsize", memsize)
        t.set_param("flopcount", flopcount)
        t.set_param("data_out", data_out)   

        tasks.append(t)
        ingest_rcal_data = data_out  # This is the data generated by the combined Ingest+RCal task; needed by subsequent tasks
        
        # Now handle the rest of the subtasks (if there are any)
        ical_task = None
        for i in range(2, nr_subtasks):
            subtask = hpso_subtasks[i]
            uid += 1
            t = SDPTask(uid, str(subtask))
            t.set_param("data_in", ingest_rcal_data) # Shared by all these tasks; needs not to be copied every time
            # TODO - replace "buffersize" as task field. Should be a 'data object' whose movement is modelled separately 
            t.set_param("memsize", performance_dict[hpso][subtask]['cache'])
            t.set_param("flopcount", performance_dict[hpso]['Tobs'] * performance_dict[hpso][subtask]['compRate'])
            
            if i == 2:
                assert subtask in HPSOs.ical_subtasks  # Assumed that this is an ical subtask
                t.set_param("prec_task", prev_ingest_task) # Associated ingest task must complete before this one can start
                ical_task = t  # Remember this task, as DPrep tasks depend on it
            elif subtask in HPSOs.dprep_subtasks:
                assert ical_task is not None
                t.set_param("prec_task", ical_task, prevent_overwrite=False)

            t.data_out  = 0  # TODO: No idea what gets output here. visRate? Temporarily set to zero 
            
            tasks.append(t)
    return tasks

def sum_deltas(deltas_history, timepoint, sorted_delta_keys=None, value_min=None, value_max=None, eps=1e-15):
    """
    Sums all the deltas chronologically up to the timestamp t
    @param deltas_history  : a dictionary that maps wall clock timestamps to a resource's value-changes
    @param timepoint       : The time until which the values should be summed
    @param sorted_delta_keys : Optional sorted timestamps; prevents re-sorting the timestamps for efficiency
    @param value_min       : Lowest allowable value for the resource's balance. Default zero.
    @param value_max       : Higest allowable value for the resource's balance. Default None.
    @param eps             : Numerical rounding tolerance
    @return                : The sum of the deltas from the beginning up to (and including) the timepoint. 
                             Returns False if value_min of value_max are violated
    """
    timestamps_sorted = sorted_delta_keys
    if timestamps_sorted is None:
        timestamps_sorted = sorted(deltas_history.keys())
    
    stop_before_index = bisect.bisect_left(timestamps_sorted, timepoint)
    if timepoint in deltas_history:
        stop_before_index += 1  # The position found by bisect needs to be included in the summation
    assert stop_before_index > 0  # The chosen time point cannot precede the first entry
        
    value_at_t = 0    
    for i in range(stop_before_index):
        value_at_t += deltas_history[timestamps_sorted[i]]
        if (((value_min is not None) and (value_at_t + eps < value_min)) or 
            ((value_max is not None) and (value_at_t - eps > value_max))):
            warnings.warn("Sum of deltas leads to value of %g at time %g sec. Outside imposed bounds of [%s,%s] " % 
                          (value_at_t, timestamps_sorted[i], value_min, value_max))
            return None
        
    return value_at_t

def find_suitable_time(deltas_history, timepoint, sorted_delta_keys=None, value_min=None, value_max=None, eps=1e-15):
    """
    Finds the smallest time >= t_min, so that the sum of the deltas is between value_min and value_max (if defined)
    @param deltas_history  : a dictionary that maps wall clock timestamps to a resource's value-changes
    @param timepoint       : The earliest point in time for the insertion
    @param sorted_delta_keys : Optional sorted timestamps; prevents re-sorting the timestamps for efficiency
    @param value_min       : Lowest allowable value of the resource's balance for insertion.
    @param value_max       : Higest allowable value of the resource's balance for insertion.
    @param eps             : Numerical rounding tolerance
    @return                : The timestamp. None, if none found.
    """
    # First cover the trivial case where there is no requirement for a suitable value at t=timepoint.
    if (value_min is None) and (value_max is None):
        return timepoint

    timestamps_sorted = sorted_delta_keys
    if timestamps_sorted is None:
        timestamps_sorted = sorted(deltas_history.keys())

    value_at_t = sum_deltas(deltas_history, timepoint, timestamps_sorted)

    # Check whether the value at the supplied timepoint is suitable. If so we just use this timepoint.
    if not (((value_min is not None) and (value_at_t + eps < value_min)) or 
            ((value_max is not None) and (value_at_t - eps > value_max))):
        return timepoint

    # Otherwise, we continue searching until we find a suitable timepoint
    print("... initial task insertion time unsuitable. Searching forward in time...")    # TODO remove
    start_at_index = bisect.bisect_left(timestamps_sorted, timepoint)
    if timepoint in deltas_history:
        start_at_index += 1  # The position found by bisect was included in the summation; start one position later
    assert start_at_index > 0
        
    for i in range(start_at_index, len(timestamps_sorted)):
        value_at_t += deltas_history[timestamps_sorted[i]]
        if not (((value_min is not None) and (value_at_t + eps < value_min)) or 
                ((value_max is not None) and (value_at_t - eps > value_max))):
            return timepoint
    
    # Otherwise, no valid timepoint has been found!
    raise Exception("No valid time point found!")
    return None


def add_delta(deltas_history, delta, t_start, t_end=None, value_min=0, value_max=None, sorted_delta_keys=None):
    """
    Applies the delta with proposed start and end times (wall clock) to a resource's simulated value change history.
    @param deltas_history  : a dictionary that maps wall clock timestamps to a resource's value-changes
    @param delta           : the change that will be added at t_start and reversed at t_end (iff t_end not None)
    @param t_start         : the wall clock time when the delta is applied
    @param t_end           : the wall clock time at which the delta expires (i.e. is reversed). Default None.
    @param value_min       : Lowest allowable value for the resource's balance. Default zero.
    @param value_max       : Higest allowable value for the resource's balance. Default None.
    @param sorted_delta_keys : Optional sorted timestamps; prevents re-sorting the timestamps for efficiency
    """    
    if t_end is not None:
        assert t_end >= t_start
    if (value_min is not None) and (value_max is not None):
        assert value_max >= value_min
        
    timestamps_sorted = sorted_delta_keys
    if timestamps_sorted is None:
        timestamps_sorted = sorted(deltas_history.keys())

    # We now insert the deltas into the variable's history, and then sum it until the end of the delta's duration
    # to ensure that we do not violate any condition by adding this delta to the history
    deltas_new = deltas_history.copy()
    
    if t_start in deltas_new:
        deltas_new[t_start] += delta
    else:
        deltas_new[t_start] = delta
        
    if t_end is not None:        
        if t_end in deltas_new:
            deltas_new[t_end] -= delta
        else:
            deltas_new[t_end] = -delta

    # The step below sums across the whole new delta sequence to make sure that it is valid. Will raise exception if not.
    timestamps_sorted = sorted(deltas_new.keys())
    value_after = sum_deltas(deltas_new, timestamps_sorted[-1], timestamps_sorted, value_min, value_max, epsilon)

    # If return value is not none the addition is valid and we can add the value into the real sequence of deltas
    if value_after is not None:
        if t_start in deltas_history:
            deltas_history[t_start] += delta
        else:
            deltas_history[t_start] = delta

        if t_end is not None:        
            if t_end in deltas_history:
                deltas_history[t_end] -= delta
            else:
                deltas_history[t_end] = -delta

## Computes  performace requirements for each HPSO using parametric model
### We do this once, and store the results in a dictionary for lookup

In [ ]:
performance_dict = {}  # A dictionary of dictionaries.  HPSO requirements are computed once and stored as lookups

# As a test we loop over all HPSOs we wish to handle, computing results for each
for task_letter in sorted(hpso_lookup.keys()):
    hpso = hpso_lookup[task_letter]
    print('*** Processing task type %s => %s ***\n' % (task_letter, hpso))
    if not hpso in performance_dict:
        performance_dict[hpso] = {}
        
    for subtask in HPSOs.hpso_subtasks[hpso]:
        print('subtask -> %s' % subtask)
        if not subtask in performance_dict[hpso]:
            performance_dict[hpso][subtask] = {}
        
        cfg = PipelineConfig(hpso=hpso, hpso_subtask=subtask)
        (valid, msgs) = cfg.is_valid()
        if not valid:
            print("Invalid configuration!")
            for msg in msgs:
                print(msg)
            raise AssertionError("Invalid config")
        tp = cfg.calc_tel_params()
        results = iapi._compute_results(cfg, False, results_map)  #TODO - refactor this method's parameter sequence
        
        performance_dict[hpso]['Tobs'] = tp.Tobs  # Observation time
        performance_dict[hpso][subtask]['ingestRate'] = results[0]
        performance_dict[hpso][subtask]['cache'] = results[1]
        performance_dict[hpso][subtask]['visRate'] = results[2]
        performance_dict[hpso][subtask]['compRate'] = results[3]
        
        print('Buffer ingest rate\t= %g TB/s' % results[0])
        print('Cache memory\t= %g TB' % results[1])
        print('Visibility IO rate\t= %g TB/s' % results[2])
        print('Compute Rate\t= %g PetaFLOP/s' % results[3])
        print()
        
print('done')

## Let's create run a short test sequence

## Now, simulate the execution of this sequence on the SDP and plot the results

In [ ]:
test_seq = ('A','A','B','B','B','A')
seqL = ('A','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','A','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B')
seqM = ('B','G','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','G','C','F','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','G','G','E','E','E','E','D')

task_list = task_letters_to_SDPTask_list(seqL, performance_dict)
for task in task_list:
    print(task)

In [ ]:
HPSO_letter_sequence = seqL
sdp_FLOPS = 22.8  # NB: The processing capacity of the SDP in PetaFLOP/s
task_list = task_letters_to_SDPTask_list(HPSO_letter_sequence, performance_dict)
max_nr_iterations = 1000  # Automatic termination if the loop fails to schedule all taks after this many iterations

# First, assert that the SDP has enough FLOP/s capacity to handle the real-time tasks. If not, we can't continue.
tasks_to_be_scheduled = set()
for task in task_list:
    tasks_to_be_scheduled.add(task)
    if task.t_fixed is not None:
        required_FLOPs = task.flopcount / task.t_fixed
        #print("Task %d requires a FLOPS rate of %g PetaFLOP/s" % (task.uid, required_FLOPs))
        if (task.flopcount / task.t_fixed) > sdp_FLOPS:
            raise AssertionError("Task %d (%s) requires %g PetaFLOP/s. SDP capacity of %g PetaFLOP/s is insufficient!" 
                                  % (task.uid, task.description, required_FLOPs, sdp_FLOPS) )
print('SDP has sufficient FLOPS capacity to handle real-time tasks.')
            
# Next, iteratively run through all tasks, scheduling them as we go along (where possible). 
# Scheduling means that each task gets put in a plausible sequence.
# Repeat until all tasks are scheduled
nr_iterations = 0
tasks_to_timestamps = {}  # Maps tasks to completion times
timestamps_to_tasks = {}  # Maps completion times to tasks lists

wall_clock = 0       # Simulated wall clock time (seconds)
wall_clock_advance = False # Iff not true, advance the wall clock to this value

flops_deltas  = {0:0}  # maps wall clock times to SDP FLOPS allocation / deallocation (+/-) sizes
memory_deltas = {0:0}  # maps wall clock times to SDP working memory (RAM) allocation / deallocation (+/-) sizes
hot_buffer_deltas = {0:0}   # maps wall clock times to hot buffer allocation / deallocation (+/-) sizes
cold_buffer_deltas = {0:0}  # maps wall clock times to cold buffer allocation / deallocation (+/-) sizes
preserve_deltas    = {0:0}  # maps wall clock times to long term preserve allocation sizes (presumably only +)

while len(tasks_to_timestamps) < len(tasks_to_be_scheduled):
    nr_iterations += 1
    print("-= Starting iteration %d =-" % nr_iterations)
    nr_tasks_scheduled_this_iteration = 0

    if nr_iterations > max_nr_iterations:
        print("Warning: Maximum number of iterations exceeded; aborting!")
        warnings.warn('Maximum number of iterations exceeded; aborting!')
        break
    
    for task in tasks_to_be_scheduled:
        if task in tasks_to_timestamps:
            continue  # The task has already been scheduled. Skipping
            
        elif (task.prec_task is not None) and (task.prec_task not in tasks_to_timestamps):
            continue
                
        elif (task.prec_task is not None) and (task.prec_task in tasks_to_timestamps) and (wall_clock < task.prec_task.t_end):
            # Wall clock is less than the end time of the preceding task (on which this task depends).
            # Therefore we may need to advance the clock to enable us to schedule any tasks! 
            if not wall_clock_advance:
                wall_clock_advance = task.prec_task.t_end
            else:
                wall_clock_advance = min(wall_clock_advance, task.prec_task.t_end)
        else:
            task_flops = 0
            if (task.t_fixed is not None):
                task_flops = task.flopcount / task.t_fixed
            else:
                flops_in_use = sum_deltas(flops_deltas, wall_clock, value_max=sdp_FLOPS)
                task_flops = max(1, (sdp_FLOPS - flops_in_use) * 0.5)  # TODO: this may be changed

            start_time = find_suitable_time(flops_deltas, wall_clock, value_max=(sdp_FLOPS-task_flops), eps=epsilon)
            t_start = start_time
            t_end = start_time + (task.flopcount / task_flops)

            if hasattr(task, "t_start"):
                raise Exception("Task already has t_start defined!\n %s" % str(task))
            task.set_param("t_start", t_start)
            task.set_param("t_end", t_end)

            add_delta(flops_deltas, task_flops, t_start, t_end, value_min=0, value_max=sdp_FLOPS)             
            add_delta(memory_deltas, task.memsize, t_start, t_end, value_min=0)             
            #add_delta(flops_deltas, flops_required, t_start, t_end, value_min=0, value_max=sdp_FLOPS)             
            #add_delta(flops_deltas, flops_required, t_start, t_end, value_min=0, value_max=sdp_FLOPS)             
            #add_delta(flops_deltas, flops_required, t_start, t_end, value_min=0, value_max=sdp_FLOPS)             

            # Add this task to the 'tasks_to_timestamps' and 'timestamps_to_tasks' mappings
            tasks_to_timestamps[task] = t_end
            if t_end in timestamps_to_tasks:
                timestamps_to_tasks[t_end].append(task)
            else:
                timestamps_to_tasks[t_end] = [task]
                
            print('* Scheduled Task %d at wall clock time %g sec. Ends at t=%g sec. ' % (task.uid, t_start, t_end))
            nr_tasks_scheduled_this_iteration += 1
        
    print('Number of scheduled tasks after %d iterations is %d out of %d' % (nr_iterations, len(tasks_to_timestamps), 
                                                                            len(tasks_to_be_scheduled)))

    if (nr_tasks_scheduled_this_iteration == 0):
        if wall_clock_advance:
            print("-> Advancing wall clock to %g sec." % wall_clock_advance)
            wall_clock = wall_clock_advance
            wall_clock_advance = False
        else:
            print("Warning! No tasks scheduled, and wall clock not advanced!")
            
'''
    # Here are the task fields, for referece: 
    uid         = None  # Unique ID - must be defined at task creation. Used for hashing and equality.
    description = None  # A human-readable description of the task
    t_min_start = None  # Earliest wall clock time (in seconds) that this task can / may start
    prec_task   = None  # Preceding task that needs to complete before this one can start (can be None)
    t_fixed     = None  # fixed minimum duration (in seconds) of this task (e.g. for an observation)
    data_in     = None  # Amount of data (in TB) this task requires to read from (presumably hot) buffer *before* starting
    memsize     = None  # The amount of SDP working memory (in TB) needed to perform this task
    flopcount   = None  # Number of operations (in PetaFLOP) required to complete this task
    data_out    = None  # Amount of data (in TB) this task stores *after* finishing (written to buffer or long-term preserve)
    data_source = None  # Data Location
    data_target = None  # Data Location
'''     
print('Done!')

In [ ]:
def plot_deltas(deltas, title="", xlabel="", ylabel=""):
    timestamps_sorted = sorted(deltas.keys())
    x_axis = []
    y_axis = []
    
    value = 0
    for t in timestamps_sorted:
        time_hours = t / 3600
        x_axis.append(time_hours)
        y_axis.append(value)
        value += deltas[t]
        x_axis.append(time_hours)
        y_axis.append(value)
    
    plt.figure()
    plt.plot(x_axis, y_axis)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(min(x_axis), max(x_axis)*1.05)
    plt.ylim(min(y_axis), max(y_axis)*1.05)

plot_deltas(flops_deltas, 'Evolution of SDP FLOP/s', 'wall clock time (hours)', 'PetaFLOP/s')
plot_deltas(memory_deltas, 'Evolution of SDP working memory (RAM)', 'wall clock time (hours)', 'TeraByte')

# Notebook incomplete beyond this point - don't execute unless you know what you're doing!

In [ ]:
t = {}
t[1] = 'a'
t[3] = 'b'
t.pop(3)
t

In [ ]:
sdp_FLOPS = 22.8  # NB: The processing capacity of the SDP in PetaFLOP/s

for task in tasks:
    if hasattr(task, )

# Run through the list of tasks, determining start and end times for their execution and the effect on the buffer

wall_clock = 0       # Simulated wall clock time (seconds)
buffer_deltas = {}   # a dictionary mapping wall clock times to buffer allocation / deallocation (+/-) sizes
t_proc_end_last = 0  # The wall clock time that the last process completed
idle_time_durations = np.zeros(len(tasks))  # in seconds
i = 0
uids_completed = set()

for task in tasks:
    task.t_obs_start = wall_clock
    add_delta(buffer_deltas, task.t_obs_start, task.bufsize)
    t_obs_end = task.t_obs_start + task.t_obs  # Time the observation completes
    task.t_proc_start = max(t_obs_end, t_proc_end_last + sdp_setup_time)
    task.t_proc_end   = task.t_proc_start + task.flopcount * task.t_obs / sdp_FLOPS
    add_delta(buffer_deltas, task.t_proc_end, -task.bufsize)
    t_proc_end_last = task.t_proc_end
    wall_clock = t_obs_end + telecope_setup_time
    idle_time_durations[i] = task.t_proc_start - t_obs_end
    i += 1

buffer_evolution = collections.OrderedDict(sorted(buffer_deltas.items()))
time_vals   = np.zeros(2 * len(buffer_evolution))
buffer_vals = np.zeros(2 * len(buffer_evolution))

i = 0
buffer_val = 0
time_val   = 0
for k, delta in buffer_evolution.items(): 
    #print('(%.1f,\t%.2f)' % (k/3600, delta))
    time_val = k / 3600  # hours
    time_vals[i]     = time_val
    buffer_vals[i]   = buffer_val
    buffer_val += delta  # Adds the buffer delta to the buffer's stored contents
    time_vals[i+1]   = time_val  # we assume no time went by (writing being instantaneous)
    buffer_vals[i+1] = buffer_val  # TeraBytes
    i += 2

plt.plot(time_vals, buffer_vals / 1e3, 'b-')
plt.title('Evolution of the SDP Buffer while executing the supplied LOW sequence.\nObservation time = %.1f hrs.' 
          ' Total execution time = %.1f hrs; Max buffer = %.1f PB' % (wall_clock / 3600, time_vals[-1],                                                                      np.max(buffer_vals)/1e3))
plt.xlabel('time (hours)')
plt.ylabel('buffer usage (PB)')
plt.xlim(0, time_vals[-1])

plt.figure()
plt.plot(np.array(range(len(idle_time_durations))), idle_time_durations / 3600, marker='s', color = 'r', linewidth=0)
plt.title('Idle time that tasks spend in the LOW buffer.\nSummed idle time for all tasks = %.1f hrs.' % 
          (np.sum(idle_time_durations) / 3600))
plt.xlabel('Task''s number in sequence')
plt.ylabel('Time (hours)')


print('Done!')

## Hard-coded performace costs and requirements from Rosie's Excel sheet
### These were previously used in rev [3372fdd] to approximately replicate Rosie's results. Check (rerun) the notebook at that repository revision to regenerate those results - not repeated here.

In [ ]:
# The following sets of values should be computed using the parametric model. Just hard-coded for now (from Excel)
hpso_ingest_rates = {'A':0.459, 'B':3e-3, 'C':0.117, 'D':0.112, 'E':0.0603, 'F':0.244, 'G':0.438}  # in TeraByte/s
# FLOPcounts below are the PetaFLOPs required to process one second of ingested data
hpso_flopcounts = {'A':50.4, 'B':2.0, 'C':7.5, 'D':6.2, 'E':2.9833, 'F':17.689, 'G':27.698}  # in PetaFLOP/s
hpso_durations  = {'A':6, 'B':0.17, 'C':6, 'D':6, 'E':4.4, 'F':0.1233, 'G':6}  # in hours -- TODO check whether correct

sdp_setup_time = 60  # the minimum amount of time between processing tasks on the SDP (seconds)
telecope_setup_time = 0  # TODO is this correct?

## Reproduction of "Low" and "Mid" sequences from Rosie's Excel sheet
### Create a lists of observation tasks as letter sequences

In [ ]:
seqL = ('A','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','A','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','A','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B')
seqM = ('B','G','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','G','C','F','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','B','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','F','G','G','E','E','E','E','D')

print('HPSO LOW task distribution (number of occurences) A..B = (%.0f, %.0f)' % (seqL.count('A'), seqL.count('B')))
tA = seqL.count('A') * hpso_durations['A']
tB = seqL.count('B') * hpso_durations['B']
print('HPSO LOW task distribution (observation time) A..B = (%.1f%%, %.1f%%)' % (100 * tA / (tA + tB), 100 * tB / (tA + tB)))

tA = seqM.count('A')
tB = seqM.count('B')
tC = seqM.count('C')
tD = seqM.count('D')
tE = seqM.count('E')
tF = seqM.count('F')
tG = seqM.count('G')
tt = len(seqM)

print('\nHPSO MID task distribution (number of occurences) A..G = (%.0f, %.0f, %.0f, %.0f, %.0f, %.0f, %.0f)' % \
      (tA, tB, tC, tD, tE, tF, tG))
print('HPSO MID task distribution (observation time) A..G = (%.1f%%, %.1f%%, %.1f%%, %.1f%%, %.1f%%, %.1f%%, %.1f%%)' % \
      (100*tA/tt, 100*tB/tt, 100*tC/tt, 100*tD/tt, 100*tE/tt, 100*tF/tt, 100*tG/tt))

### Use the lists of letters to build a lists of task objects

### Virtually execute the "seqL" task list for LOW

In [ ]:
tasks = task_letters_to_objects(seqL, performance_dict)  # Set up the task list from the letter sequence

sdp_FLOPS = 22.8  # NB: The processing capacity of the SDP in PetaFLOP/s

# Run through the list of tasks, determining start and end times for their execution and the effect on the buffer

wall_clock = 0       # Simulated wall clock time (seconds)
buffer_deltas = {}   # a dictionary mapping wall clock times to buffer allocation / deallocation (+/-) sizes
t_proc_end_last = 0  # The wall clock time that the last process completed
idle_time_durations = np.zeros(len(tasks))  # in seconds
i = 0
uids_completed = set()

for task in tasks:
    task.t_obs_start = wall_clock
    add_delta(buffer_deltas, task.t_obs_start, task.bufsize)
    t_obs_end = task.t_obs_start + task.t_obs  # Time the observation completes
    task.t_proc_start = max(t_obs_end, t_proc_end_last + sdp_setup_time)
    task.t_proc_end   = task.t_proc_start + task.flopcount * task.t_obs / sdp_FLOPS
    add_delta(buffer_deltas, task.t_proc_end, -task.bufsize)
    t_proc_end_last = task.t_proc_end
    wall_clock = t_obs_end + telecope_setup_time
    idle_time_durations[i] = task.t_proc_start - t_obs_end
    i += 1

buffer_evolution = collections.OrderedDict(sorted(buffer_deltas.items()))
time_vals   = np.zeros(2 * len(buffer_evolution))
buffer_vals = np.zeros(2 * len(buffer_evolution))

i = 0
buffer_val = 0
time_val   = 0
for k, delta in buffer_evolution.items(): 
    #print('(%.1f,\t%.2f)' % (k/3600, delta))
    time_val = k / 3600  # hours
    time_vals[i]     = time_val
    buffer_vals[i]   = buffer_val
    buffer_val += delta  # Adds the buffer delta to the buffer's stored contents
    time_vals[i+1]   = time_val  # we assume no time went by (writing being instantaneous)
    buffer_vals[i+1] = buffer_val  # TeraBytes
    i += 2

plt.plot(time_vals, buffer_vals / 1e3, 'b-')
plt.title('Evolution of the SDP Buffer while executing the supplied LOW sequence.\nObservation time = %.1f hrs.' 
          ' Total execution time = %.1f hrs; Max buffer = %.1f PB' % (wall_clock / 3600, time_vals[-1],                                                                      np.max(buffer_vals)/1e3))
plt.xlabel('time (hours)')
plt.ylabel('buffer usage (PB)')
plt.xlim(0, time_vals[-1])

plt.figure()
plt.plot(np.array(range(len(idle_time_durations))), idle_time_durations / 3600, marker='s', color = 'r', linewidth=0)
plt.title('Idle time that tasks spend in the LOW buffer.\nSummed idle time for all tasks = %.1f hrs.' % 
          (np.sum(idle_time_durations) / 3600))
plt.xlabel('Task''s number in sequence')
plt.ylabel('Time (hours)')


print('Done!')

# Scratchpad

In [ ]:
#               Table Row Title            Unit     Default?  Sum?   Expression
results_map =[('Total buffer ingest rate','TeraBytes/s',True, False, lambda tp: tp.Rvis_ingest*tp.Nbeam*tp.Npp*tp.Mvis/c.tera),
              ('Working (cache) memory',  'TeraBytes',  True, True,  lambda tp: tp.Mw_cache/c.tera,   ),
              ('Visibility I/O Rate',     'TeraBytes/s',True, True,  lambda tp: tp.Rio/c.tera,        ),
              ('Total Compute Req',       'PetaFLOP/s', True, True,  lambda tp: tp.Rflop/c.peta,      ),
              ('Comp Req Breakdown ->',   'PetaFLOP/s', True, True,  lambda tp: tp.get_products('Rflop', scale=c.peta), )]
del results_map[4]  # We actually don't care about the breakdown for now; but it is useful to know how to get it

hpso = hpso_lookup['A']  # hpso01.ICAL

cfg = PipelineConfig(hpso=hpso)
assert cfg.is_valid()
tp = cfg.calc_tel_params()

results = iapi._compute_results(cfg, False, results_map)  #TODO - refactor this method's parameter sequence
print('Cache memory for hpso01.ICAL = %g TB' % results[1])
print('Visibility rate for hpso01.ICAL = %g TB/s' % results[2])
print('Rflop for hpso01.ICAL = %g PetaFLOPS' % results[3])

# Another, slightly more roundabout, way to do the same as _compute_results 
# (tsnap_opt, nfacet_opt) = imp.find_optimal_Tsnap_Nfacet(tp)
# result_expressions = iapi.get_result_expressions(results_map, tp)
# results_for_pipeline = imp.evaluate_expressions(result_expressions, tp, tsnap_opt, nfacet_opt)
# print(results_for_pipeline[3])

## Example code taken from computing parametric model results by Pipeline

In [ ]:
teles = (Telescopes.SKA1_Low, Telescopes.SKA1_Mid)
bands = (Bands.Low, 
         Bands.Mid1, Bands.Mid2, Bands.Mid5A, Bands.Mid5B, Bands.Mid5C,
         Bands.Sur1)
parallel = 0  # Set this to 0 if PyMP is absent

for pipeline in Pipelines.all:
    iapi.stack_bars_pipelines("%s Computational Requirements [PetaFLOP/s]" % pipeline, teles, bands, [pipeline],
                              parallel=parallel)

In [ ]:
for band in bands:
    iapi.stack_bars_pipelines("%s Computational Requirements [PetaFLOP/s]" % band, teles, [band], Pipelines.all,
                              parallel = parallel)

In [ ]:
iapi.stack_bars_hpsos("HPSOs Computational Requirements [PetaFLOP/s]", HPSOs.hpsos,
                      parallel=16)